In [ ]:
# importing required libraries
import sys
import numpy as numpy
from pandas import read_csv, DataFrame
import matplotlib.pyplot as pyplot
! pip install scikit-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# libraries for ARIMA model
! pip install pmdarima
import pmdarima as pmdarima
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

# libraries for LSTM model
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping

# Importing and pre-processing data

In [ ]:
# array of lags that will be analysed
lags_arr = [24, 48, 72]

# array of token symbols that will be analysed
token_symbols = ['UNI', 'LINK', 'AAVE', 'MKR', 'LEO', 'COMP', 'GRT', 'HT', 'CEL', 
                 'CHZ', 'TEL', 'YFI', 'HOT', 'ENJ', 'MANA', 'QNT', 'BAT', 'SNX', 'NEXO', 
                 'BNT', 'CRV', 'CHSB', 'KCS', 'ZRX', 'UMA', 'ANKR', 'VGX', '1INCH']
number_of_tokens = len(token_symbols)

# importing data from csv files (before executing this step, csv data files should be uploaded)
raw_datasets = []
for symbol in token_symbols:
  raw_datasets.append(read_csv('{}.csv'.format(symbol)))

In [ ]:
# scaling data and splitting datasets to training and test datasets
training_set_ratio = 0.80
data_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Trades']
scalers = []
training_datasets, test_datasets = [], []

for i in range(number_of_tokens):
  # separating dates from values, converting data to float type
  raw_dataset_without_dates = raw_datasets[i][data_columns].astype('float')

  # scaling data
  scaler = MinMaxScaler()
  training_set_size = int(len(raw_datasets[i])*training_set_ratio)
  raw_training_dataset = raw_dataset_without_dates[:training_set_size]
  scaler.fit(raw_training_dataset)
  scaled_dataset = DataFrame(scaler.transform(raw_dataset_without_dates), columns=data_columns)
  scalers.append(scaler)

  # splitting scaled dataset to training and test datasets
  training_datasets.append(scaled_dataset[:training_set_size])
  test_datasets.append(scaled_dataset[training_set_size:])

#ARIMA

In [ ]:
# determining optimal ARIMA parameters for each dataset
ARIMA_params = []
for i in range(number_of_tokens):
  model = pmdarima.auto_arima(training_datasets[i]['Close'], 
                        test='adf',             # using Augmented Dickey-Fuller (ADF) test to find optimal 'd'
                        d=None,                 # stating that the model should determine 'd' parameter
                        start_p=1, max_p=3,     # setting range for potential 'p' parameters
                        start_q=1, max_q=3,     # setting range for potential 'q' parameters
                        error_action='ignore',  # stating that erros should not be printed
                        suppress_warnings=True) # stating that warnings should not be printed
  
  # recording calculated optimal ARIMA parameters
  ARIMA_params.append(model.order)
  
  # printing out the parameters for visibility
  print(token_symbols[i] + ' - ' + str(model.order))

In [ ]:
# stating that warnings should not be printed
import warnings
warnings.filterwarnings("ignore")

# predicting future prices with ARIMA
ARIMA_predictions_for_all_lags = []

# for each lags option (i.e. for 24 lags, 48 lags, and 72 lags)
for lags in lags_arr:
  print('{} lags:'.format(lags))
  ARIMA_predictions = []  

  # for each token
  for i in range(number_of_tokens):
    # Extracting  closing prices data
    test_dataset_close_prices = test_datasets[i]['Close'].to_numpy().tolist()
    training_dataset_close_prices = training_datasets[i]['Close'].to_numpy().tolist()

    # creating batches for testing
    training_X = []
    for j in range(len(test_dataset_close_prices)):
      curr_data = []
      if j >= lags:
        curr_data.extend(test_dataset_close_prices[(j-lags):j])
      else:
        curr_data.extend(training_dataset_close_prices[-(lags-j):])
        curr_data.extend(test_dataset_close_prices[0:j])
      training_X.append(curr_data)

    # forecasting future prices from the test batches with ARIMA
    curr_predictions = []
    for j in range(len(test_dataset_close_prices)):
      sys.stdout.write('\r' + '{} - {}/{}'.format(token_symbols[i], j, len(test_dataset_close_prices)))
      sys.stdout.flush()
      model = ARIMA(training_X[j], order=ARIMA_params[i])
      model_fit = model.fit()
      output = model_fit.forecast()
      yhat = output[0]
      curr_predictions.append(yhat)
    
    # recording predictions for a token
    ARIMA_predictions.append(curr_predictions)
    print(' - done\n')

  # recording predictions for all tokens
  ARIMA_predictions_for_all_lags.append(ARIMA_predictions)

#LSTM

In [ ]:
# creating batches for training and testing
training_y_arr, training_X_arr, test_X_arr = [], [], []

for lags in lags_arr:
  # arrays for storing batches for trianing and testing
  training_X, test_X = [], []

  # array for storing dependent variables of the training dataset
  training_y = []

  for i in range(number_of_tokens):
    # extracting dependent variables of the training dataset
    training_y.append(training_datasets[i][lags:]['Close'].to_numpy())

    # creating batches for training
    curr_training_X = []
    for j in range(0, len(training_datasets[i]) - lags):
      curr_training_X.append(training_datasets[i][j:j+lags].to_numpy().tolist())
    training_X.append(numpy.array(curr_training_X))

    # creating batches for testing
    curr_test_X = []
    for j in range(len(test_datasets[i])):
      curr_data = []
      if j >= lags:
        curr_data.extend(test_datasets[i][(j-lags):j].to_numpy().tolist())
      else:
        curr_data.extend(training_datasets[i][-(lags-j):].to_numpy().tolist())
        curr_data.extend(test_datasets[i][0:j].to_numpy().tolist())
      curr_test_X.append(curr_data)
    test_X.append(numpy.array(curr_test_X))

  # recording dependent variables of the training datase
  training_y_arr.append(training_y)

  # recording training and testing batches
  training_X_arr.append(training_X)
  test_X_arr.append(test_X)

In [ ]:
# array for storing all LSTM predictions results
LSTM_predictions_for_all_lags = []

# for each lag option
for l in range(len(lags_arr)):
  LSTM_predictions = []
  print('{} lags:'.format(lags_arr[l]))

  # for each token building LSTM models and predicting test values
  for i in range(number_of_tokens):
    # building an LSTM model
    training_data = training_X_arr[l][i]
    model = Sequential()
    model.add(LSTM(64, input_shape=(training_data.shape[1], training_data.shape[2]), return_sequences=False))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')

    # training the LSTM model with the test dataset (test batches and dependent variables of the training datase)
    earlyStopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    model.fit(training_data, training_y_arr[l][i], epochs=1000, batch_size=64,
                        validation_split=0.9, verbose=2, shuffle=False, callbacks=[earlyStopping])
    
    # making predictions by the LSTM model
    yhat = model.predict(test_X_arr[l][i])

    # recording predictions
    LSTM_predictions.append(yhat.flatten('C'))
    print('{} - done\n\n'.format(token_symbols[i]))

  # recording all predictions
  LSTM_predictions_for_all_lags.append(LSTM_predictions)

#Calculate performance of ARIMA and LSTM

In [ ]:
def mean_absolute_percentage_error(test_data, predicted_data): 
    return numpy.mean(numpy.abs((test_data - predicted_data) / test_data)) * 100

def calculate_performance(predictions, test_data, scaler):
  # rescaling the predictions to normal scale
  dataframe_for_rescaling = DataFrame({"col1":predictions, "col2":predictions, "col3":predictions, 
                                       "col4":predictions, "col5":predictions, "col6":predictions})
  dataframe_scaled_back = scaler.inverse_transform(dataframe_for_rescaling)
  predictions_scaled_back = DataFrame(dataframe_scaled_back)[3].to_numpy()

  # rescaling test dataset back to normal scale
  dataframe_for_rescaling = DataFrame({"col1":test_data, "col2":test_data, "col3":test_data, 
                                       "col4":test_data, "col5":test_data, "col6":test_data})
  dataframe_scaled_back = scaler.inverse_transform(dataframe_for_rescaling)
  test_y_scaled_back = DataFrame(dataframe_scaled_back)[3].to_numpy()

  # calculating performance metrics
  RMSE = mean_squared_error(test_y_scaled_back, predictions_scaled_back, squared=False) # Root mean square error
  MAE =  mean_absolute_error(test_y_scaled_back, predictions_scaled_back) # Mean absolute error
  MAPE = mean_absolute_percentage_error(test_y_scaled_back, predictions_scaled_back) # Mean absolute percentage error
  return [RMSE, MAE, MAPE]

In [ ]:
# array holding all results
all_results = []

# calculating and saving performance results for each lag option
for l in range(len(lags_arr)):
  lags = lags_arr[l]
  print('{} lags:'.format(lags))

  results = DataFrame(columns=['Token', 'ARIMA_RMSE','LSTM_RMSE', 'ARIMA_MAE', 'LSTM_MAE', 'ARIMA_MAPE', 'LSTM_MAPE'])

  # calculating and saving performance results for each token
  for i in range(number_of_tokens):
    # calculating performance for a token
    test_data = test_datasets[i]['Close'].to_numpy()
    ARIMA_performance = calculate_performance(ARIMA_predictions_for_all_lags[l][i], test_data, scalers[i])
    LSTM_performance = calculate_performance(LSTM_predictions_for_all_lags[l][i], test_data, scalers[i])

    # recording results for a token
    results = results.append({'Token': token_symbols[i], 
                              'ARIMA_RMSE': ARIMA_performance[0], 'LSTM_RMSE': LSTM_performance[0],
                              'ARIMA_MAE': ARIMA_performance[1], 'LSTM_MAE': LSTM_performance[1], 
                              'ARIMA_MAPE': ARIMA_performance[2], 'LSTM_MAPE': LSTM_performance[2]}, 
                            ignore_index=True)
  
  # recording all results
  all_results.append(results)
  print(results)
  print()

In [ ]:
# downloading results as a file (optional)
from google.colab import drive
from google.colab import files
drive.mount('/drive')

for i in range(len(lags_arr)):
  fileName = 'results_ARIMA_LSTM_{}_lags.csv'.format(lags_arr[i])
  all_results[i].to_csv(fileName)
  files.download(fileName)

Mounted at /drive


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# a method for plotting results
def plotting_results(token_symbols, ARIMA_series, LSTM_series, metrics_name, number_of_lags):
  # locations of the labels
  x = numpy.arange(len(token_symbols))

  # width of a bars
  width = 0.35

  # passing data for the chart
  fig, ax = pyplot.subplots()
  series1 = ax.bar(x - width/2, ARIMA_series, width, label='ARIMA')
  series2 = ax.bar(x + width/2, LSTM_series, width, label='LSTM')

  # adding labels, title, and formatting
  ax.set_ylabel('Scaled logarithms of {}'.format(metrics_name))
  ax.set_title('{} results for {} lags'.format(metrics_name, number_of_lags))
  ax.set_xticks(x)
  ax.set_xticklabels(token_symbols)
  ax.legend()
  pyplot.rcParams["figure.figsize"] = (12,3)
  pyplot.xticks(rotation = 90)
  pyplot.show()

In [ ]:
# plotting charts with results for each lag option
for i in range(len(all_results)):
  # selecting list of tokens from the results
  token_symbols = all_results[i]['Token']
  number_of_tokens = len(token_symbols)

  # concatenating ARIMA and LSTM results for scaling for better data visualisation
  arima = all_results[i][['ARIMA_RMSE', 'ARIMA_MAE', 'ARIMA_MAPE']].to_numpy()
  lstm = all_results[i][['LSTM_RMSE', 'LSTM_MAE', 'LSTM_MAPE']].to_numpy()
  concatendated_results = DataFrame(numpy.concatenate((arima, lstm)), columns=['RMSE', 'MAE', 'MAPE'])

  # computing logarithm of the results, so the scale is less diverge
  log_results = numpy.log(concatendated_results)

  # scaling the results data so it can be visualised on the same chart
  scaler = MinMaxScaler()
  scaledResults = DataFrame(scaler.fit_transform(log_results), columns=['RMSE', 'MAE', 'MAPE'])

  # separating ARIMA and LSTM scaled results
  ARIMA_scaled_results = scaledResults[:number_of_tokens]
  LSTM_scaled_results = scaledResults[number_of_tokens:]

  # Plotting charts
  print('Results for {} lags:'.format(lags_arr[i]))
  plotting_results(token_symbols, ARIMA_scaled_results['RMSE'], LSTM_scaled_results['RMSE'], 'RMSE', lags_arr[i])
  plotting_results(token_symbols, ARIMA_scaled_results['MAE'], LSTM_scaled_results['MAE'], 'MAE', lags_arr[i])
  plotting_results(token_symbols, ARIMA_scaled_results['MAPE'], LSTM_scaled_results['MAPE'], 'MAPE', lags_arr[i])
  print('\n\n')